# [추천 시스템] -> 유사도 찾기
- TF-IDF와 코사인 유사도로 추천 시스템 구현
- 원리 : 유사한 내용 추천으로 유사도가 높은 것 찾기
    * 영화 줄거리(overview)가 비슷한 영화를 찾아주기

# 영화추천
- 데이터 준비

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# 데이터 준비
datafile = '../data/movies_metadata.csv'

dataDF1 = pd.read_csv(datafile, low_memory=False) 
'''
low_memory 
대용량의 데이터를 불러오는 경우 각 칼럼의 데이터 타입(dtype)을 추측하는 것이 매우 많은 메모리를 사용하기 때문에 대용량의 데이터를 불러올때 메모리 에러가 발생하는 경우 이를 False로 설정할 것을 권장한다.
'''
dataDF1.info()
dataDF1.head(n=2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [8]:
# 10000개 데이터만 사용
dataDF2 = dataDF1.head(10000)[["id", "title", "overview"]]
dataDF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10000 non-null  object
 1   title     10000 non-null  object
 2   overview  9971 non-null   object
dtypes: object(3)
memory usage: 234.5+ KB


# 2. 데이터 전처리
- 기본 데이터 전처리
- 결측치는 overview에 대해서만 하면됨

In [10]:
# 결측치
dataDF2["overview"].isnull().sum()

29

In [12]:
dataDF2 = dataDF2.dropna(subset=['overview'])
dataDF2["overview"].isnull().sum()

0

In [15]:
dataDF2.reset_index(drop=True, inplace=True)
dataDF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9971 entries, 0 to 9970
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        9971 non-null   object
 1   title     9971 non-null   object
 2   overview  9971 non-null   object
dtypes: object(3)
memory usage: 233.8+ KB


# 3. TF-IDF와 consine_similarity

In [16]:
# TF-IDF : 단어들의 값 계산
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(dataDF2['overview'])

# cosine 유사도 : 두개 matrix에 대한 비교 진행
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [17]:
tfidf_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
print(f"TF-IDF: {tfidf_matrix.shape}")
print(f"코사인 유사도: {cosine_sim.shape}")

TF-IDF: (9971, 32350)
코사인 유사도: (9971, 9971)


In [19]:
cosine_sim[:10], dataDF2.loc[:10, "title"]

(array([[1.        , 0.01682702, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.01682702, 1.        , 0.04871756, ..., 0.        , 0.01200553,
         0.        ],
        [0.        , 0.04871756, 1.        , ..., 0.        , 0.00734935,
         0.        ],
        ...,
        [0.        , 0.        , 0.00686199, ..., 0.01933384, 0.        ,
         0.        ],
        [0.        , 0.1071591 , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 0                       Toy Story
 1                         Jumanji
 2                Grumpier Old Men
 3               Waiting to Exhale
 4     Father of the Bride Part II
 5                            Heat
 6                         Sabrina
 7                    Tom and Huck
 8                    Sudden Death
 9                       GoldenEye
 10         The American President
 Name: title, dtype: obj

In [21]:
# 영화 제목 => 해당 영화 인덱스 추출
(dataDF2["title"] == "Father of the Bride Part II").argmax()

4

In [23]:
# 영화제목 : 인덱스 => 딕셔너리
title_to_index = dict(zip(dataDF2["title"], dataDF2.index))

# 원하는 영화 인덱스 찾기
select_idx = title_to_index["Father of the Bride Part II"]
select_idx

4

In [28]:
# 모든 영화 유사도
sim_scores = list(enumerate(cosine_sim[select_idx]))
print(sim_scores)

# 유사도에 따라 정렬
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
print(sim_scores)

# 가장 유사한 10개
sim_scores = sim_scores[1:11]
movie_indices = [idx[0] for idx in sim_scores]

# 가장 유사한 10개의 영화의 제목
dataDF1[["title", "genres"]].iloc[movie_indices]

[(0, 0.0), (1, 0.0), (2, 0.025777101022163285), (3, 0.0), (4, 1.0), (5, 0.0), (6, 0.03421029274686588), (7, 0.0), (8, 0.032819482334982074), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.01631159814518899), (17, 0.0), (18, 0.023797468860159075), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.01033383039523807), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.013065267556380763), (33, 0.0), (34, 0.0), (35, 0.009003353345639568), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.013332788381309739), (41, 0.00523029015961616), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0270353404382027), (47, 0.0), (48, 0.0), (49, 0.030664280409498104), (50, 0.00896961124875655), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.01789628699919174), (56, 0.0), (57, 0.019136436195432475), (58, 0.0), (59, 0.016217919896723387), (60, 0.013720845896029336), (61, 0.0), (62, 0.013132508478050052), (63, 0.0), (64, 0.

,title,genres
6769,The Hired Hand,"[{'id': 37, 'name': 'Western'}]"
6547,Bollywood/Hollywood,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
6282,Ziggy Stardust and the Spiders from Mars,"[{'id': 99, 'name': 'Documentary'}, {'id': 104..."
4984,Cast a Giant Shadow,"[{'id': 10752, 'name': 'War'}, {'id': 28, 'nam..."
7073,No Good Deed,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
914,The Mark of Zorro,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
5550,Take Care of My Cat,"[{'id': 18, 'name': 'Drama'}]"
5728,They All Laughed,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name..."
1500,When the Cat's Away,"[{'id': 35, 'name': 'Comedy'}]"
6789,In My Skin,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name..."


In [29]:
import torch

torch.__version__

'2.2.0'